In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split

def getWifiInfoAndSort(wifistrs):
    transform = lambda wifiinfo: (wifiinfo[0], int(wifiinfo[1]), wifiinfo[2] == "True")
    return sorted([transform(onewifi.split('|')) for onewifi in wifistrs.split(";")], key=lambda x:x[1] ,reverse=True) # sorted by db

def scoreForWifi(shopDatas, k, unconnectWeights, connectWeight):
    wifiToShop = defaultdict(lambda :defaultdict(lambda :0)) # wifiToShop[wifi][shop]
    for rowSeries in shopDatas.iterrows():
        wifiForOneShop = getWifiInfoAndSort(rowSeries[1]['wifi_infos'])
        wifiForOneShop = wifiForOneShop[:min(len(wifiForOneShop),k)]
        currentShop = rowSeries[1]['shop_id']
        for idx, wifi in enumerate(wifiForOneShop):
            if not wifi[2]:
                wifiToShop[wifi[0]][currentShop] += unconnectWeights[idx] 
            else:
                wifiToShop[wifi[0]][currentShop] += connectWeight  #if wifi is connected
    return wifiToShop

def getShop(wifiToShop, wifis, k, weights): # 获取前k个最强的wifi信号
    shopScore = defaultdict(lambda :0) # wifiToShop[wifi][shop]
    wifiForOneShop = getWifiInfoAndSort(wifis)
    wifiForOneShop = wifiForOneShop[:min(len(wifiForOneShop),k)]
    for idx, wifi in enumerate(wifiForOneShop):
        for shop in wifiToShop[wifi[0]]:
            shopScore[shop] += weights[idx] + wifiToShop[wifi][shop] # 给当前搜索到的wifi根据强度赋一个权重 然后乘上对应wifi对应商店的权重 
            #当前越近的wifi影响力越大
    if len(shopScore) == 0:
        return "Nan"
    return sorted(shopScore.items(), key=lambda x: x[1], reverse=True)[0][0]

In [2]:
# load Data
#user_id	shop_id	time_stamp	longitude	latitude	wifi_infos
userBehavior = pd.read_csv("/home/szh-921/workSpace/data/train-ccf_first_round_user_shop_behavior.csv")
testDataSet = pd.read_csv("/home/szh-921/workSpace/data/AB-testSet-evaluation_public.csv")


trainDataSet = userBehavior[['wifi_infos','shop_id']]

#xtrain, xtest, ytrain, ytest = train_test_split(trainDataSet['wifi_infos'], trainDataSet['shop_id'], test_size=0.25, random_state=33)
#trainDataSet = pd.DataFrame({'wifi_infos':xtrain, 'shop_id':ytrain})
#testDataSet = pd.DataFrame({'row_id':np.arange(ytest.shape[0]), 'wifi_infos':xtest})
testDataSet = testDataSet[['row_id', 'wifi_infos']]

In [3]:
#traingData = pd.DataFrame({'wifi_info': wifiTrain, 'shop_id': shopTrain})
#testData = pd.DataFrame({'wifi_info': wifiTest, 'shop_id': shopTest})#.head(200)
#print(type(testData.values))

In [4]:
# main
# parameter set
k = 20
weights = list(reversed(range(51)))
testK = k
trainK = k

In [5]:
#Traing
wifiToShop = scoreForWifi(trainDataSet, trainK, weights[:k], k)

In [6]:
#Test
result = ['0,1'] * testDataSet.shape[0]
#print(len(result))
#print(testDataSet.shape)
for idx, row in enumerate(testDataSet.iterrows()):
    result[int(idx)] = getShop(wifiToShop, row[1]['wifi_infos'], testK, weights[:k])

In [7]:
# ouput result
tosubmmitResult = pd.DataFrame({'row_id': testDataSet['row_id'], 'shop_id': result})
#acc = (ytest == tosubmmitResult['shop_id']).sum() / testDataSet.shape[0]
#print("acc=", acc)
tosubmmitResult.to_csv("/home/szh-921/workSpace/data/submmit_result.csv")